In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit, when, rand, current_date, date_add
import os
from datetime import datetime, timedelta

spark = SparkSession.builder.appName("SyntheticData") \
    .config("spark.master", "local") \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.cores', '4') \
    .config('spark.executor.memory', '8g') \
    .getOrCreate()

# Функция получает от цикла дату с нечетными днями
def function(current_date_str):
    current_date_str = current_date_str.strftime("%Y-%m-%d")
    # Генерация данных для колонок id, name, city, email, age, salary, registration_date
    data = [(i, f"Names_{i}", f"Cityname_{i}") for i in range(1, 101)]
    df = spark.createDataFrame(data, ["id", "name", "city"])
    
    df = df.withColumn("email", concat(col("name"), lit("@example."), when(col("id") % 2 == 0, "ru").otherwise("com"))) \
        .withColumn("age", (rand() * 78 + 18).cast("int")) \
        .withColumn("salary", (rand() * 150000 + 19242).cast("int")) \
        .withColumn("registration_date", date_add(current_date(), - (rand() * (col("age") - 18)).cast("int") * 365))
    
    df.cache()  # Кеширование DataFrame для более быстрой обработки
    
    # Замена 5% данных на значение NULL
    columns = ["name", "email", "city", "age", "salary", "registration_date"]
    df = df.select("id", *[when(rand() <= 0.05, None).otherwise(col(column)).alias(column) for column in columns])
    
    # Путь к сохранению CSV файла
    path = f"/home/jovyan/work/PySpark_test/config/temp/{current_date_str}-dev.csv"
    path_rename = f"/home/jovyan/work/PySpark_test/config/more_csv/{current_date_str}-dev.csv"
    
    # Создание директории, если она не существует
    directory = "/home/jovyan/work/PySpark_test/config/more_csv/"
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Запись DataFrame в CSV
    df.coalesce(1).write.csv(path, header=True, mode="overwrite")  
    
    # Переименование сгенерированного файла
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if file.startswith("part-"):
            os.rename(file_path, path_rename)
    
    # Удаление временных файлов и папки
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        os.remove(file_path)
        
    os.rmdir(path)
    
# Указываем первую дату
current_date_str = datetime(2024, 5, 1)

# Указываем количество дней
amount_of_day = 60

# Цикл вызывает функцию когда находит дату с нечетными днями
num = 0
while num < amount_of_day:
    if current_date_str.day % 2 != 0:
        function(current_date_str)
    current_date_str += timedelta(days=1)
    num += 1

print("Код выполнен")
spark.stop()

Код выполнен


In [2]:
##########################################################################
#  Блок кода для шифрования
import os
from cryptography.fernet import Fernet

# Создание директории, если она не существует
directory = '/home/jovyan/work/PySpark_test/General_task_№2/utils/'
if not os.path.exists(directory):
    os.makedirs(directory)

# Директория ключа
path_key = f"/home/jovyan/work/PySpark_test/General_task_№2/utils/"

# Генерация ключа и сохранение в файл
key = Fernet.generate_key()
with open(f"{path_key}secret.key", "wb") as key_file:
    key_file.write(key)

cipher_suite = Fernet(key)

# Шифрование данных
def encrypt_file(file_path):
    with open(file_path, 'rb') as file:
        original_data = file.read()
    encrypted_data = cipher_suite.encrypt(original_data)
    with open(file_path, 'wb') as file:
        file.write(encrypted_data)

# Функция находит все файлы в директории и вызывает функцию шифрования данных
def encrypt_directory(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            encrypt_file(file_path)

# Вызов функции для нужной директории
path_to_encrypt = "/home/jovyan/work/PySpark_test/config/more_csv"
encrypt_directory(path_to_encrypt)

print("Шифрование завершено")
##########################################################################

Шифрование завершено


In [3]:
##########################################################################
# Блок кода для дешифрования
import os
from cryptography.fernet import Fernet

# Директория ключа
path_key = "/home/jovyan/work/PySpark_test/General_task_№2/utils/"

# Открываем ключ
with open(f"{path_key}secret.key", "rb") as key_file:
    key = key_file.read()

f = Fernet(key)

# Дешифрование данных
def decrypt(file_path):
    with open(file_path, 'rb') as file:
        encrypted_data = file.read()

    try:
        decrypted_data = f.decrypt(encrypted_data)
    except cryptography.fernet.InvalidToken:
        print("Недопустимый токен")
        return
    
    with open(file_path, 'wb') as file:
        file.write(decrypted_data)

# Функция находит все файлы в директории и вызывает функцию дешифрования данных
def crypt_directory(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            decrypt(file_path)

# Вызов функции для нужной директории
path_to_decrypt = "/home/jovyan/work/PySpark_test/config/more_csv"
crypt_directory(path_to_decrypt)

print("Дешифрование завершено")
##########################################################################

Дешифрование завершено
